In [154]:
import tensorflow as tf
import numpy as np
import pandas as pd


In [155]:
import chess
from chess import pgn

In [156]:
from chess import pgn

def games_in(file):
    """
    Returns the list of games from a pgn file
    """
    numGames = 0
    games = open(file)
    game = pgn.read_game(games)
    games_list = []
    while game is not None and numGames < 300:
        games_list.append(game)
        game = pgn.read_game(games)
        numGames += 1
    return games_list

In [157]:
from os import listdir
from os.path import isfile, join

def files_in(path):
    return [join(path,f) for f in listdir(path) if isfile(join(path, f))]

In [158]:
PIECES = chess.PIECE_TYPES

In [159]:
curpath = "lichessdb"

In [160]:
files = files_in(curpath)

In [161]:
gameslist = []
for file in files:
    gameslist.extend(games_in(file))

In [162]:
columns = ['wMate', 'bMate', 'draw', 'turn', 
           'wP','wN','wB','wR','wQ','wK','bP','bN','bB','bR','bQ','bK',
           'hwP','hwN','hwB','hwR','hwQ','hwK','hbP','hbN','hbB','hbR','hbQ','hbK',
          'EVAL']
def boardfeatures(board):
    """
    Feature containing castling rights, turn, and every possible piece type belonging in every possible square
    """
    featuresList = []
    featuresList.append(currentTurn(board))
    featuresList.extend(castlingRights(board))
    featuresList.extend(PList(containsPCinSquare, board))
    return featuresList
    

def features(board):
    """
    Returns a list of numerical values for features of a board. Subject to change.
    [ischeckmate[W/B], isDraw, turn, material[PNBRKQ][W/B] ,hangingMaterial[W/b]]
    """
    featuresList = []
    featuresList.extend(colorList(isCheckmate, board))
    featuresList.extend([isDraw(board)])
    featuresList.extend([currentTurn(board)])
    featuresList.extend(pieceColorList(numPieceOfColor, board))
    featuresList.extend(pieceColorList(numHangingPieceOfColor, board))
    return featuresList
    
def containsPCinSquare(board, piece, color, square):
    if board.color_at(square) == color and board.piece_type_at(square) == piece:
        return 1
    return 0

def PCSList(fn, board, piece, color):
    a = [fn(board, piece, color, square) for square in chess.SQUARES]
    return a

def PCList(fn, board, piece):
    a = PCSList(fn, board, piece, chess.WHITE)
    b = PCSList(fn, board, piece, chess.BLACK)
    a.extend(b)
    return a

def PList(fn, board):

    lists = [PCList(fn, board, piece) for piece in chess.PIECE_TYPES]
    answer = []
    for i in lists:
        answer.extend(i);
    return answer
    
def castlingRights(board):
    return [int(board.has_kingside_castling_rights(chess.WHITE)), 
           int(board.has_queenside_castling_rights(chess.WHITE)),
           int(board.has_kingside_castling_rights(chess.BLACK)),
           int(board.has_queenside_castling_rights(chess.BLACK))]


def pieceColorList(fn, board):
    """
    Returns a list containing FN called on every piece type and color combo
    """
    a = [fn(board, piece, chess.WHITE) for piece in PIECES]
    a.extend([fn(board, piece, chess.BLACK) for piece in PIECES])
    return a
    
def colorList(fn, board):
    """
    Returns a list containing FN called on both color
    """
    a = [fn(board, chess.WHITE)]
    a.extend([fn(board, chess.BLACK)])
    return a
    
def isCheckmate(board, color):
    """
    Returns 1 if the current color is checkmating the other
    """
    if board.is_checkmate() and board.turn == (not color):
        return 1
    return 0

def isDraw(board):
    """
    Returns 1 iff the current position is a draw
    """
    if board.result() == '1/2-1/2': #might have to input some * in result
        return 1
    return 0

def currentTurn(board):
    if board.turn == chess.WHITE:
        return 1
    return 0

def numPieceOfColor(board, piece, color):
    """
    Returns the number of pieces of specific color
    """
    return len(board.pieces(piece, color))

def numHangingPieceOfColor(board, piece, color): 
    #Update function to still count pieces as hanging if num attacker pawns > defender pawns, repeat for each piece...
    """
    Returns the number of hanging pieces of a specific color 
    """
    our_pieces = board.pieces(piece, color)
    opp_color = not color
    hanging_pieces = 0
    for sq in our_pieces:
        unpinned_attackers_on_sq = sum([not board.is_pinned(not color, opp_att) for opp_att in board.attackers(opp_color, sq)])
        unpinned_defenders_on_sq = sum([not board.is_pinned(color, opp_att) for opp_att in board.attackers(color, sq)]) 
        if unpinned_attackers_on_sq > unpinned_defenders_on_sq:
            hanging_pieces += 1
    return hanging_pieces
        

In [163]:
testBoard = gameslist[0].board()
testBoard.push(chess.Move.from_uci('e2e4'))
testBoard.push(chess.Move.from_uci('e7e5'))
testBoard.push(chess.Move.from_uci('d1h5'))
testBoard.push(chess.Move.from_uci('g8f6'))
testBoard.push(chess.Move.from_uci('f1c4'))
testBoard.push(chess.Move.from_uci('b8c6'))
testBoard.push(chess.Move.from_uci('h5f7'))
print(testBoard)

print(boardfeatures(testBoard))

r . b q k b . r
p p p p . Q p p
. . n . . n . .
. . . . p . . .
. . B . P . . .
. . . . . . . .
P P P P . P P P
R N B . K . N R
[0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0

In [164]:
def length_of_game(game):
    return sum(1 for e in game.mainline_moves())

def result(game):
    result = game.headers["Result"]
    answer = 0
    if result == '1-0':
        answer = 1
    elif result == '0-1':
        answer = -1
    return answer

In [165]:
def add_game_to_dataset(array, game):
    board = game.board()
    result = game.headers["Result"]
    multiplier = 0
    if result == '1-0':
        multiplier = 1
    elif result == '0-1':
        multiplier = -1
    movenumber = 0
    if multiplier != 0:
        for move in game.mainline_moves():
            current_move += 1
            board.push(move)
            if movenumber > 10:
                current_features = boardfeatures(board)
                current_features.append(multiplier)
                array.append(current_features)

In [166]:
import random
def games_to_dataset(games):
    """
    def add_to_datadict(features, result):
        tupfeatures = tuple(features)
        if(data_dict.get(tupfeatures) is not None):
            data = data_dict.get(tupfeatures)
        else:
            data = (0,0)
        data = list(data)
        data[0] = data[0] + result
        data[1] = data[1] + 1
        data = tuple(data)
        data_dict[tupfeatures] = data
        """
    white_dataset = []
    black_dataset = []
    for game in games:
        board = game.board()
        pts = result(game)
        gamelength = length_of_game(game)
        curmove = 0
        if (pts == 1 or pts == -1) and gamelength > 25:
            randommoves = random.sample(range(10, gamelength - 1), 5)
            for move in game.mainline_moves():
                curmove += 1
                board.push(move)
                if curmove in randommoves:
                    current_features = boardfeatures(board)
                    current_features.append(pts)
                    if pts == 1:
                        white_dataset.append(current_features)
                    else:
                        black_dataset.append(current_features)

    white_dataset = white_dataset[:150]
    black_dataset = black_dataset[:150]
    return white_dataset + black_dataset
    
        

            

In [167]:
#IMPORT THE DATASET

modelgameslist = gameslist[:100]
dataset = games_to_dataset(modelgameslist)
data = pd.DataFrame(dataset)
x = data.iloc[:,:-1].values
y = data.iloc[:,-1].values
data

,0,1,2,3,4,5,6,7,8,9,...,764,765,766,767,768,769,770,771,772,773
0,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
6,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
7,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
8,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
9,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [168]:
#SPLIT TRAIN AND TEST SET
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [169]:
#FEATURE SCALING
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
x_train = mms.fit_transform(x_train)
x_test = mms.transform(x_test)
x_train

array([[1., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [170]:
#BUILDING THE ANN
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=773, activation='relu'))
ann.add(tf.keras.layers.Dense(units=100, activation='relu'))
ann.add(tf.keras.layers.Dense(units=10, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='linear'))

In [171]:
#TRAINING THE ANN
ann.compile(optimizer='adam', loss='mean_squared_error')
ann.fit(x_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
8/8 [==============================] - 0s 3ms/step - loss: 0.8913
Epoch 2/100
8/8 [==============================] - 0s 3ms/step - loss: 0.3468
Epoch 3/100
8/8 [==============================] - 0s 3ms/step - loss: 0.1694
Epoch 4/100
8/8 [==============================] - 0s 3ms/step - loss: 0.1109
Epoch 5/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0503
Epoch 6/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0270
Epoch 7/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0154
Epoch 8/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0078
Epoch 9/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0043
Epoch 10/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0023
Epoch 11/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0020
Epoch 12/100
8/8 [==============================] - 0s 3ms/step - loss: 0.0017
Epoch 13/100
8/8 [==============================] - 0s 3ms/st

In [180]:
testBoard = gameslist[0].board()
testBoard.push(chess.Move.from_uci('e2e4'))
testBoard.push(chess.Move.from_uci('e7e5'))
testBoard.push(chess.Move.from_uci('d1h5'))
testBoard.push(chess.Move.from_uci('g8f6'))
testBoard.push(chess.Move.from_uci('f1c4'))
testBoard.push(chess.Move.from_uci('f6h5'))
testarray = [boardfeatures(testBoard)]
testarray = mms.transform(testarray)
print(testarray)
ann.predict(testarray)


[[1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.
  1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

array([[1.0049055]], dtype=float32)